In [6]:
import cv2 as cv
import dlib
import numpy as np
import time
import pygame
from imutils import face_utils
from scipy.spatial import distance as dist

#定义一些变量
#无法检测到人脸次数
NO_FACE_COUNT = 0
#眼睛张合度阈值
EYE_OPEN_SIZE = 0.2
#嘴巴张合度阈值
MOUSE_OPEN_SIZE = 1
#用以疲劳驾驶提示
TIRED_PROMPT = 0
#用以记录疲劳提示次数
PROMPT_COUNT = 0
#头部偏移度阈值
HEAD_ANGLE_SIZE = 15
#记录疲劳提示时间
PROMPT_TIME = np.zeros((1, 999))

object_pts = np.float32([[6.825897, 6.760612, 4.402142], 
                         [1.330353, 7.122144, 6.903745],  
                         [-1.330353, 7.122144, 6.903745], 
                         [-6.825897, 6.760612, 4.402142], 
                         [5.311432, 5.485328, 3.987654], 
                         [1.789930, 5.393625, 4.413414],  
                         [-1.789930, 5.393625, 4.413414],
                         [-5.311432, 5.485328, 3.987654],
                         [2.005628, 1.409845, 6.165652],  
                         [-2.005628, 1.409845, 6.165652], 
                         [2.774015, -2.080775, 5.048531], 
                         [-2.774015, -2.080775, 5.048531],
                         [0.000000, -3.116408, 6.097667], 
                         [0.000000, -7.415691, 4.070434]])
K = [6.5308391993466671e+002, 0.0, 3.1950000000000000e+002,
     0.0, 6.5308391993466671e+002, 2.3950000000000000e+002,
     0.0, 0.0, 1.0]
D = [7.0834633684407095e-002, 6.9140193737175351e-002, 0.0, 0.0, -1.3073460323689292e+000]
cam_matrix = np.array(K).reshape(3, 3).astype(np.float32)
dist_coeffs = np.array(D).reshape(5, 1).astype(np.float32)
reprojectsrc = np.float32([[10.0, 10.0, 10.0],
                           [10.0, 10.0, -10.0],
                           [10.0, -10.0, -10.0],
                           [10.0, -10.0, 10.0],
                           [-10.0, 10.0, 10.0],
                           [-10.0, 10.0, -10.0],
                           [-10.0, -10.0, -10.0],
                           [-10.0, -10.0, 10.0]])

#计算眼睛张合度
def eye_open(left, right):
    #计算左眼张合度
    lh1 = dist.euclidean(left[1], left[5])
    lh2 = dist.euclidean(left[2], left[4])
    lw = dist.euclidean(left[0], left[3])
    l_open = (lh1 + lh2) / (2.0 * lw)
    #计算右眼张合度
    rh1 = dist.euclidean(right[1], right[5])
    rh2 = dist.euclidean(right[2], right[4])
    rw = dist.euclidean(right[0], right[3])
    r_open = (rh1 + rh2) / (2.0 * rw)
    #计算眼睛的张合度（左右眼的张合度取平均）
    eye_open_level = (l_open + r_open) / 2.0
    return eye_open_level

#计算嘴巴张合度
def mouse_open(mouse):
    h1 = np.linalg.norm(mouse[2] - mouse[10])
    h2 = np.linalg.norm(mouse[4] - mouse[8])
    w = np.linalg.norm(mouse[0] - mouse[6])
    mouse_open_level = (h1 + h2) / (2.0 * w)
    return mouse_open_level

#计算头部偏移度
def head_pose(obj):
    image_pts = np.float32([obj[17], obj[21], obj[22], obj[26], obj[36],obj[39], obj[42], obj[45], obj[31], obj[35],
                            obj[48], obj[54], obj[57], obj[8]])
    _, rotation_vec, translation_vec = cv.solvePnP(object_pts, image_pts, cam_matrix, dist_coeffs)
    reprojectdst, _ = cv.projectPoints(reprojectsrc, rotation_vec, translation_vec, cam_matrix,dist_coeffs)
    reprojectdst = tuple(map(tuple, reprojectdst.reshape(8, 2)))
    # 计算欧拉角calc euler angle
    #罗德里格斯公式（将旋转矩阵转换为旋转向量）
    rotation_mat, _ = cv.Rodrigues(rotation_vec)
    pose_mat = cv.hconcat((rotation_mat, translation_vec))
    _, _, _, _, _, _, euler_angle = cv.decomposeProjectionMatrix(pose_mat)
    return reprojectdst, euler_angle
    
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(".data/shape_predictor_68_face_landmarks.dat")
font = cv.FONT_HERSHEY_PLAIN
cap = cv.VideoCapture(0)
isOpen = cap.isOpened()

while isOpen:
    k = cv.waitKey(10)
    _, frame = cap.read()
    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    faces = detector(gray, 0)
    #左眼坐标点
    left_eye_point = []
    #右眼坐标点
    right_eye_point = []
    #嘴巴坐标点
    mouth_point = []
    #鼻子坐标点
    nose_point = []
    
    if len(faces) == 0:
        NO_FACE_COUNT += 1
    else:
        #未检测到人脸的情况中断时，阈值回到0
        NO_FACE_COUNT = 0
        for face in faces:
            landmark = predictor(gray, face)
            #landmark.parts() 68个人脸特征点
            #所有特征点坐标提取
            landmarks  = np.matrix([[point.x, point.y] for point in landmark.parts()])
        #获取左眼、右眼、嘴巴及鼻子坐标
        left_eye_point = landmarks[36:42]
        right_eye_point = landmarks[42:48]
        mouse_point = landmarks[48:]
        nose_point = landmarks[27:36]
        #时间计算人眼闭眼时间（小于所设的眼睛张合度EYE_OPEN_SIZE）
        end_time = time.time()
        '''for i in range(len(left_eye_point)):
            cv.circle(frame, (left_eye_point[i,0], left_eye_point[i,1]), 1, (255, 255, 0))
            cv.circle(frame, (right_eye_point[i,0], right_eye_point[i,1]), 1, (255, 255, 0))
        for i in range(len(mouse_point)):
            cv.circle(frame, (mouse_point[i,0], mouse_point[i,1]), 1, (255, 255, 0))'''
        #眼睛张合度
        eye_open_level = eye_open(left_eye_point, right_eye_point)
        #嘴巴张合度
        mouse_open_level = mouse_open(mouse_point)
        #头部姿势
        reprojectdst, head_angle = head_pose(landmarks)
        
        cv.putText(frame, 'EyeOpenLevel:' + str(round(eye_open_level, 2)), (10,20), font, 1, (255, 0, 0))
        cv.putText(frame, 'MouseOpenLevel:' + str(round(mouse_open_level, 2)), (10,40), font, 1, (255, 0, 0))
        cv.putText(frame, 'HeadAngle_pitch:' + str(round(head_angle[0, 0], 2)), (10,60), font, 1, (255, 0, 0))
        cv.putText(frame, 'HeadAngle_roll:' + str(round(head_angle[2, 0], 2)), (10,80), font, 1, (255, 0, 0))
        
        #小于眼睛张合度阈值，视为眼睛疲劳状态
        if eye_open_level < EYE_OPEN_SIZE:
            pass
        else:
            eye_start_time = time.time()
            
        #大于嘴巴张合度阈值
        if mouse_open_level > MOUSE_OPEN_SIZE:
            pass
        else:
            mouse_start_time = time.time()
        
        #头部偏移度过大及眼睛张合度低于阈值
        if ((head_angle[0, 0] > HEAD_ANGLE_SIZE ) and (eye_open_level < EYE_OPEN_SIZE)) or ((abs(head_angle[2, 0]) > HEAD_ANGLE_SIZE ) and (eye_open_level < EYE_OPEN_SIZE)):
            pass
        else:
            head_start_time = time.time()
        
        #判断闭眼时间是否持续两秒以上
        if (end_time - eye_start_time) > 2:
            now = time.time()
            #两次疲劳提示时间间隔大于10秒
            if (now - PROMPT_TIME[0][PROMPT_COUNT]) > 10:
                #播报语音提示闭眼时间过长
                #print('Eye closed too long')
                file = r'./data/EyeClose.mp3'
                pygame.mixer.init()
                track = pygame.mixer.music.load(file)
                #播放语音提示
                pygame.mixer.music.play()
                
                PROMPT_COUNT += 1
                TIRED_PROMPT += 1
                #记录当前疲劳提示时间
                PROMPT_TIME[0][PROMPT_COUNT] = now
                
        #判断张嘴时间是否持续两秒以上
        if (end_time - mouse_start_time) > 2:
            now = time.time()
            if (now - PROMPT_TIME[0][PROMPT_COUNT]) > 10:
                #播报语音提示打哈欠
                file = r'./data/Yawn.mp3'
                pygame.mixer.init()
                track = pygame.mixer.music.load(file)
                #播放语音提示
                pygame.mixer.music.play()
                
                PROMPT_COUNT += 1
                TIRED_PROMPT += 1
                #记录当前疲劳提示时间
                PROMPT_TIME[0][PROMPT_COUNT] = now
            
        #判断犯困点头
        if (end_time - head_start_time) > 0.5:
            #播报语音提示请注意休息
            file = r'./Voice/Tired.mp3'
            pygame.mixer.init()
            track = pygame.mixer.music.load(file)
            #播放语音提示
            pygame.mixer.music.play()
        
    #50帧监测不到人脸，提示回到驾驶位置
    if NO_FACE_COUNT == 50:
        file = r'./data/Back.mp3'
        pygame.mixer.init()
        track = pygame.mixer.music.load(file)
        #播放语音提示
        pygame.mixer.music.play()

        NO_FACE_TIME = 0
    #疲劳提示达到三次，播报语音提示休息
    if TIRED_PROMPT == 3:
        file = r'./data/Tired.mp3'
        pygame.mixer.init()
        track = pygame.mixer.music.load(file)
        #播放语音提示
        pygame.mixer.music.play()
        
        TIRED_PROMPT = 0
    
    cv.imshow("Frame", frame)
    #按Ese键退出
    key = cv.waitKey(1)
    if key == 27:
        break
#释放窗口，关闭摄像头
cap.release()
cv.destroyAllWindows()

ModuleNotFoundError: No module named 'pygame'